In [2]:
import json
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

# datasets is a list of available datasets descriptions containing: path, key columns names, and suitable complex network features
from src.data.dataset_info import datasets


In [3]:
dataset = datasets[3]
name = dataset.name
print("dataset: {}".format(name))

path = "./datasets/partitions/{}.pkl".format(name)
new_path = "./datasets/preprocessed/{}.pkl".format(name)
graph_path = "./datasets/preprocessed/graph_{}.gexf".format(name)
df = pd.read_pickle(path)

dataset: partition_1


In [ ]:
# converting all infinity values into nan then dropping all records containing nan values
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(axis=0, how='any', inplace=True)

df.drop_duplicates(subset=list(set(df.columns) - set([dataset.timestamp_col, dataset.flow_id_col])), keep="first", inplace=True)

In [ ]:
total_count = len(df)

properties = {
    "name": dataset.name,
    "length": total_count,
}

num_benign = len(df[df['Label'] == 0])
num_attack = len(df[df['Label'] == 1])

properties["num_benign"] = num_benign
properties["percentage_of_benign_records"] = ((num_benign * 100)/total_count)

properties["num_attack"] = num_attack
properties["percentage_of_attack_records"] = ((num_attack * 100)/total_count)

properties["attacks"] = list(df["Attack"].unique())  # .to_list()

filename = ('./datasets_properties/{}.json'.format(dataset.name))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(properties))
outfile.close()

In [ ]:
G = nx.from_pandas_edgelist(
        df,
        source=dataset.src_ip_col,
        target=dataset.dst_ip_col,
        create_using=nx.DiGraph()
    )

In [ ]:
G.remove_nodes_from(list(nx.isolates(G)))

In [ ]:
import igraph as ig
G1 = ig.Graph.from_networkx(G)
part = G1.community_infomap()

communities = []
for com in part:
    communities.append([G1.vs[node_index]['_nx_name'] for node_index in com])

print(f"==>> number of communities: {len(communities)}")
for com in communities:
    print(f"==>> com: {len(com)}")

In [ ]:
properties = {}

properties["number_of_nodes"] = G.number_of_nodes()
properties["number_of_edges"] = G.number_of_edges()

degrees = [degree for _, degree in G.degree()]
properties["max_degree"] = max(degrees)
properties["avg_degree"] = sum(degrees) / len(degrees)

In [ ]:
properties["transitivity"] = nx.transitivity(G)


In [ ]:
properties["density"] =  nx.density(G)


In [ ]:
# Assuming G is your graph and communities is a list of sets, where each set contains the nodes in a community

# Step 1: Map each node to its community
node_to_community = {}
for community_index, community in enumerate(communities):
    for node in community:
        node_to_community[node] = community_index

# Step 2: Count inter-cluster edges efficiently
inter_cluster_edges = 0
for u, v in G.edges():
    # Directly check if u and v belong to different communities
    if node_to_community[u] != node_to_community[v]:
        inter_cluster_edges += 1


properties["mixing_parameter"] = inter_cluster_edges / G.number_of_edges()

In [ ]:
properties["modularity"] = nx.community.modularity(G, communities)

In [ ]:
filename = ('./datasets_properties/{}.json'.format("graph_" + name))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(properties))
outfile.close()

properties

In [ ]:
community_labels = {}
for i, community in enumerate(communities):
    for node in community:
        community_labels[node] = i

nx.set_node_attributes(G, community_labels, "new_community")

In [ ]:
# getting inter and itra graph, to calculate the local and global variations of each centrality
from src.network.network_features import separate_graph

intra_graph, inter_graph = separate_graph(G, communities)

In [ ]:
from src.network.network_features import cal_betweenness_centrality

if "betweenness" in dataset.cn_measures:
    nx.set_node_attributes(G, cal_betweenness_centrality(G), "betweenness")
    print("calculated")

In [ ]:
if "local_betweenness" in dataset.cn_measures:
    nx.set_node_attributes(G, cal_betweenness_centrality(intra_graph), "local_betweenness")
    print("calculated")

In [ ]:
if "global_betweenness" in dataset.cn_measures:
    nx.set_node_attributes(G, cal_betweenness_centrality(inter_graph), "global_betweenness")
    print("calculated")

In [ ]:
if "degree" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.degree_centrality(G), "degree")
    print("calculated")

In [ ]:
if "local_degree" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.degree_centrality(intra_graph), "local_degree")
    print("calculated")

In [ ]:
if "global_degree" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.degree_centrality(inter_graph), "global_degree")
    print("calculated")

In [ ]:
if "eigenvector" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.eigenvector_centrality(G, max_iter=600), "eigenvector")
    print("calculated")


In [ ]:
if "local_eigenvector" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.eigenvector_centrality(intra_graph), "local_eigenvector")
    print("calculated")

In [ ]:
if "global_eigenvector" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.eigenvector_centrality(inter_graph), "global_eigenvector")
    print("calculated")

In [ ]:
if "closeness" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.closeness_centrality(G), "closeness")
    print("calculated")

In [ ]:
if "local_closeness" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.closeness_centrality(intra_graph), "local_closeness")
    print("calculated")

In [ ]:
if "global_closeness" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.closeness_centrality(inter_graph), "global_closeness")
    print("calculated")

In [ ]:
if "pagerank" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.pagerank(G, alpha=0.85), "pagerank")
    print("calculated")

In [ ]:
if "local_pagerank" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.pagerank(intra_graph, alpha=0.85), "local_pagerank")
    print("calculated")

In [ ]:
if "global_pagerank" in dataset.cn_measures:
    nx.set_node_attributes(G, nx.pagerank(inter_graph, alpha=0.85), "global_pagerank")
    print("calculated")

In [ ]:
from src.network.network_features import cal_k_core

if "k_core" in dataset.cn_measures:
    nx.set_node_attributes(G, cal_k_core(G), "k_core")
    print("calculated")

In [ ]:
from src.network.network_features import cal_k_truss
if "k_truss" in dataset.cn_measures:
    nx.set_node_attributes(G, cal_k_truss(G), "k_truss")
    print("calculated")

In [ ]:
from src.network.CommCentralityCode import comm_centreality

if "Comm" in dataset.cn_measures:
    nx.set_node_attributes(G, comm_centreality(G, community_labels), "Comm")
    print("calculated")

In [ ]:
from src.network.modularity_vitality import modularity_vitality

if "mv" in dataset.cn_measures:
    nx.set_node_attributes(G, modularity_vitality(G1, part), "mv")
    print("calculated")

In [ ]:
nx.write_gexf(G, graph_path)

In [ ]:
"""
features_dicts = {}
for measure in dataset.cn_measures:
    features_dicts[measure] = nx.get_node_attributes(G, measure)
    print(f"==>> features_dicts: {measure , len(features_dicts[measure])}")
    
for feature in dataset.network_features:
        if feature[:3] == "src":
            df[feature] = df.apply(
                lambda row: features_dicts[feature[4:]].get(row[dataset.src_ip_col], -1), axis=1)
        if feature[:3] == "dst":
            df[feature] = df.apply(
                lambda row: features_dicts[feature[4:]].get(row[dataset.dst_ip_col], -1), axis=1)
"""

In [ ]:
#df.head()

In [ ]:
#pd.to_pickle(df, new_path)

In [ ]:
#from unfied_features import create_unified_features

In [9]:
dataset = datasets[3]
name = dataset.name
print("dataset: {}".format(name))

new_path = "./datasets/preprocessed/{}.pkl".format(name)
graph_path = "./datasets/preprocessed/graph_{}.gexf".format(name)
G = nx.read_gexf(graph_path)



dataset: partition_1


In [11]:
import pickle
import pandas as pd

# Assuming 'name' is defined as in your context
new_path = "./datasets/preprocessed/{}.pkl".format(name)

# Load the dataset from the .pkl file
with open(new_path, 'rb') as f:
    data = pickle.load(f)

# Get all column names
if isinstance(data, pd.DataFrame):
    columns = data.columns.tolist()
    print(f"Total columns: {len(columns)}")
    print("All column names:")
    print(columns)
    
    # For better visualization of many columns
    print("\nFormatted column list:")
    for i, col in enumerate(columns, 1):
        print(f"{i:3}. {col}")
        
else:
    print("Data is not a DataFrame. Actual type:", type(data))
    if hasattr(data, 'columns'):  # Check for other objects with columns
        print("Alternative columns:", data.columns)
    elif hasattr(data, 'keys'):  # Handle dictionaries
        print("Dictionary keys:", list(data.keys())[:106])

Total columns: 106
All column names:
['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg'

In [ ]:
"""network2_features = create_unified_features(G, dataset)
print(network2_features.shape)
print(network2_features[:5]) """ # Slicing to get the first 5 rows


In [13]:
from autoencoder import extract_features, build_autoencoder
import numpy as np
from tensorflow.keras import layers, Model

In [14]:

network2_features = extract_features(G, dataset.cn_measures)


In [15]:
# Store input dimension
input_dim = len(dataset.cn_measures)  # Input dimension
# Build and train the autoencoder
autoencoder = build_autoencoder(input_dim)
autoencoder.fit(network2_features, network2_features, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
1939/1939 [==============================] - 3s 1ms/step - loss: 0.1631 - val_loss: 0.0447
Epoch 2/100
1939/1939 [==============================] - 3s 1ms/step - loss: 0.0426 - val_loss: 0.0413
Epoch 3/100
1939/1939 [==============================] - 3s 1ms/step - loss: 0.0416 - val_loss: 0.0412
Epoch 4/100
1939/1939 [==============================] - 3s 1ms/step - loss: 0.0416 - val_loss: 0.0412
Epoch 5/100
1939/1939 [==============================] - 3s 1ms/step - loss: 0.0416 - val_loss: 0.0412
Epoch 6/100
1939/1939 [==============================] - 3s 1ms/step - loss: 0.0416 - val_loss: 0.0412
Epoch 7/100
1939/1939 [==============================] - 3s 1ms/step - loss: 0.0415 - val_loss: 0.0412
Epoch 8/100
1939/1939 [==============================] - 3s 1ms/step - loss: 0.0415 - val_loss: 0.0412
Epoch 9/100
1939/1939 [==============================] - 3s 1ms/step - loss: 0.0415 - val_loss: 0.0412
Epoch 10/100
1939/1939 [==============================] - 3s 1ms/step - l

In [16]:
# Extract latent vectors
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.layers[1].output)
latent_vectors = encoder.predict(network2_features)

# Print latent vectors shape
print("Latent vectors shape:", latent_vectors.shape)

2423/2423 [==============================] - 2s 846us/step
Latent vectors shape: (77530, 8)


In [24]:
print(latent_vectors[:12])# Print the first 5 rows
len(latent_vectors)

[[1.7945313e+00 0.0000000e+00 3.8656294e-03 0.0000000e+00 5.5582601e-01
  3.2793155e-01 0.0000000e+00 7.5755221e-01]
 [1.8609260e+00 0.0000000e+00 2.4941714e-01 0.0000000e+00 4.3229628e-01
  3.5838300e-01 0.0000000e+00 7.7774745e-01]
 [1.7897533e+00 0.0000000e+00 7.0978701e-04 0.0000000e+00 5.3963012e-01
  3.2724354e-01 0.0000000e+00 7.4095494e-01]
 [1.7945710e+00 0.0000000e+00 3.8901865e-03 0.0000000e+00 5.5593383e-01
  3.2794032e-01 0.0000000e+00 7.5762999e-01]
 [1.8612460e+00 0.0000000e+00 2.4996580e-01 0.0000000e+00 4.3769318e-01
  3.5274291e-01 0.0000000e+00 7.9228544e-01]
 [1.7945710e+00 0.0000000e+00 3.8901865e-03 0.0000000e+00 5.5593383e-01
  3.2794032e-01 0.0000000e+00 7.5762999e-01]
 [1.7945510e+00 0.0000000e+00 3.8747489e-03 0.0000000e+00 5.5590785e-01
  3.2793725e-01 0.0000000e+00 7.5765496e-01]
 [1.7945710e+00 0.0000000e+00 3.8901865e-03 0.0000000e+00 5.5593383e-01
  3.2794032e-01 0.0000000e+00 7.5762999e-01]
 [1.7945504e+00 0.0000000e+00 3.8742572e-03 0.0000000e+00 5.5590

77530